In [4]:
import pandas as pd
import plotly.graph_objects as go
import json, random
import numpy as np

In [5]:
main_df = pd.read_csv('survey_results_public.csv')
#main_df.head(3)

In [59]:
main_df.columns

Index(['ResponseId', 'Q120', 'MainBranch', 'Age', 'Employment', 'RemoteWork',
       'CodingActivities', 'EdLevel', 'LearnCode', 'LearnCodeOnline',
       'LearnCodeCoursesCert', 'YearsCode', 'YearsCodePro', 'DevType',
       'OrgSize', 'PurchaseInfluence', 'TechList', 'BuyNewTool', 'Country',
       'Currency', 'CompTotal', 'LanguageHaveWorkedWith',
       'LanguageWantToWorkWith', 'DatabaseHaveWorkedWith',
       'DatabaseWantToWorkWith', 'PlatformHaveWorkedWith',
       'PlatformWantToWorkWith', 'WebframeHaveWorkedWith',
       'WebframeWantToWorkWith', 'MiscTechHaveWorkedWith',
       'MiscTechWantToWorkWith', 'ToolsTechHaveWorkedWith',
       'ToolsTechWantToWorkWith', 'NEWCollabToolsHaveWorkedWith',
       'NEWCollabToolsWantToWorkWith', 'OpSysPersonal use',
       'OpSysProfessional use', 'OfficeStackAsyncHaveWorkedWith',
       'OfficeStackAsyncWantToWorkWith', 'OfficeStackSyncHaveWorkedWith',
       'OfficeStackSyncWantToWorkWith', 'AISearchHaveWorkedWith',
       'AISearchWan

In [144]:
main_df['AIDevWantToWorkWith']

0                   NaN
1        GitHub Copilot
2                   NaN
3                   NaN
4                   NaN
              ...      
89179         Whispr AI
89180               NaN
89181               NaN
89182               NaN
89183    GitHub Copilot
Name: AIDevWantToWorkWith, Length: 89184, dtype: object

## radial chart data

In [66]:
import pandas as pd

# Assuming 'main_df' is your DataFrame
df = main_df[['AIBen']]

# Create a new DataFrame with the counts of each unique value in 'AIBen'
count_df = df['AIBen'].value_counts().reset_index()

# Rename the columns for clarity
count_df.columns = ['AIBen', 'Count']

# Create a new column with the percentage weightage
count_df['Percentage'] = (count_df['Count'] / count_df['Count'].sum()) * 100

# Merge the count DataFrame with the original DataFrame on the 'AIBen' column
result_df = pd.merge(df, count_df, on='AIBen', how='left')

# Round the 'Percentage' column to two decimal places
result_df['Percentage'] = result_df['Percentage'].round(2)

# Drop the 'Count' column
result_df = result_df.drop('Count', axis=1)

# Add a new column 'Value' with values '0' and '270' repeated for each row
result_df['Value'] = ['0', '270'] * (len(result_df) // 2)

# If the number of rows is odd, add an additional '0' for the last row
if len(result_df) % 2 != 0:
    result_df.loc[len(result_df) - 1, 'Value'] = '0'

# Drop the count_df DataFrame
del count_df
count_df=result_df.dropna().drop_duplicates().copy()

count_df.to_csv('radial.csv', index=False)


# chord diagram

In [68]:
import pandas as pd

# Create DataFrame
df = main_df[['AISearchHaveWorkedWith', 'AISearchWantToWorkWith']].copy()

# Split and explode the values in the columns
df['AISearchHaveWorkedWith'] = df['AISearchHaveWorkedWith'].str.split(';')
df['AISearchWantToWorkWith'] = df['AISearchWantToWorkWith'].str.split(';')

df = df.explode('AISearchHaveWorkedWith')
df = df.explode('AISearchWantToWorkWith').dropna()


# Pivot the DataFrame to create a matrix
matrix = df.pivot_table(index='AISearchHaveWorkedWith', columns='AISearchWantToWorkWith', aggfunc='size', fill_value=0)


# Convert the DataFrame to JSON
#json_data = matrix.to_json(orient='index')

# Display the JSON data
#print(json_data)

# sankey diagram

In [155]:
import pandas as pd

# Create DataFrame
df = main_df[['AIDevHaveWorkedWith','AIDevWantToWorkWith']].copy()

# Split and explode the values in the columns
df['AIDevHaveWorkedWith'] = df['AIDevHaveWorkedWith'].str.split(';')
df['AIDevWantToWorkWith'] = df['AIDevWantToWorkWith'].str.split(';')

df = df.explode('AIDevHaveWorkedWith')
df = df.explode('AIDevWantToWorkWith').dropna()


# Pivot the DataFrame to create a matrix
ai_tool_matrix = df.pivot_table(index='AIDevHaveWorkedWith', columns='AIDevWantToWorkWith', aggfunc='size', fill_value=0)


# Convert the DataFrame to JSON
#json_data = matrix.to_json(orient='index')

# Display the JSON data
#print(json_data)

In [156]:
# Create a Size column
df['Size'] = 1

# Group by Source, Target, and sum the Size
sankey_df = df.groupby(['AIDevHaveWorkedWith', 'AIDevWantToWorkWith']).agg({'Size': 'sum'}).reset_index()

# Drop rows where Source and Target are the same
sankey_df = sankey_df[sankey_df['AIDevHaveWorkedWith'] != sankey_df['AIDevWantToWorkWith']]
sankey_df.to_csv('sankey_df.csv')


In [160]:
import plotly.graph_objects as go

# Assuming your DataFrame is named sankey_df
sankey_df 

# Get unique nodes
nodes = sankey_df['AIDevHaveWorkedWith'].append(sankey_df['AIDevWantToWorkWith']).unique()

# Create a figure
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=nodes
    ),
    link=dict(
        source=sankey_df['AIDevHaveWorkedWith'].apply(lambda x: nodes.tolist().index(x)),
        target=sankey_df['AIDevWantToWorkWith'].apply(lambda x: nodes.tolist().index(x)),
        value=sankey_df['Size']
    )
)])

fig.update_layout(title_text="AIDev Collaboration Flows", font_size=10)
fig.show()


AttributeError: 'Series' object has no attribute 'append'

In [146]:
ai_tool_matrix

AIDevWantToWorkWith,AWS CodeWhisperer,Adrenaline,Codeium,GitHub Copilot,Mintlify,Replit Ghostwriter,Rubber Duck.AI,Synk Code,Tabnine,Whispr AI
AIDevHaveWorkedWith,,,,,,,,,,
AWS CodeWhisperer,1091,62,79,1122,65,104,81,84,162,106
Adrenaline,49,88,29,69,46,44,45,44,52,53
Codeium,82,32,328,207,33,40,38,34,62,43
GitHub Copilot,2231,191,342,15966,172,417,281,346,1020,622
Mintlify,51,38,28,105,110,39,38,41,55,43
Replit Ghostwriter,74,38,28,196,39,145,40,41,75,50
Rubber Duck.AI,49,42,30,72,37,39,73,40,44,43
Synk Code,92,46,38,269,47,49,46,279,85,55
Tabnine,763,86,154,2782,96,166,100,140,1897,189


In [154]:
# Reset index
ai_tool_matrix_reset = ai_tool_matrix.reset_index()

# Melt the DataFrame
melted_df = ai_tool_matrix_reset.melt(id_vars='AIDevHaveWorkedWith', var_name='AIDevWantToWorkWith', value_name='Size')

# Filter non-zero values
melted_df = melted_df[melted_df['Size'] > 0]
melted_df

melted_df.to_csv('sankey_df.csv')


In [129]:
import pandas as pd

df1 = main_df[['AIToolInterested in Using']].copy()

# Split and explode the values in the columns
df1['Using'] = df1['AIToolInterested in Using'].str.split(';')
df1 = df1.explode('Using').dropna()
df1['Ai Tool'] = 'Intrested In Using'
df1 = df1.drop('AIToolInterested in Using', axis=1)
matrix1 = df1.pivot_table(index='Ai Tool', columns='Using', aggfunc='size', fill_value=0)




df2 = main_df[['AIToolCurrently Using']].copy()

# Split and explode the values in the columns
df2['Using'] = df2['AIToolCurrently Using'].str.split(';')
df2 = df2.explode('Using').dropna()
df2['Ai Tool'] = 'Currently Using'
df2 = df2.drop('AIToolCurrently Using', axis=1)


matrix2= df2.pivot_table(index='Ai Tool', columns='Using', aggfunc='size', fill_value=0)




df3 = main_df[['AIToolNot interested in Using']].copy()

# Split and explode the values in the columns
df3['Using'] = df3['AIToolNot interested in Using'].str.split(';')
df3 = df3.explode('Using').dropna()
df3['Ai Tool'] = 'Not Intrested Using'
df3 = df3.drop('AIToolNot interested in Using', axis=1)

matrix3= df3.pivot_table(index='Ai Tool', columns='Using', aggfunc='size', fill_value=0)



concatenated_df = pd.concat([matrix1, matrix2,matrix3], axis=0, ignore_index=False)
#concatenated_df.to_csv('merged_aitools_cnct1.csv')
concatenated_df

Using,Collaborating with teammates,Committing and reviewing code,Debugging and getting help,Deployment and monitoring,Documenting code,Learning about a codebase,Other (please describe),Project planning,Testing code,Writing code
Ai Tool,,,,,,,,,,
Intrested In Using,11305,18670,15335,17137,18945,18467,286,14534,20807,8945
Currently Using,1377,3806,18437,1788,12963,11350,579,5097,9000,31131
Not Intrested Using,15606,8654,2401,10682,3042,4936,301,11227,4316,1690


In [110]:
import pandas as pd

def process_aitool_column(df, column_name, tool_category):
    df_temp = df[[column_name]].copy()
    df_temp['Using'] = df_temp[column_name].str.split(';')
    df_temp = df_temp.explode('Using').dropna()
    df_temp['Ai Tool'] = tool_category
    df_temp = df_temp.drop(column_name, axis=1)
    matrix = df_temp.pivot_table(index='Ai Tool', columns='Using', aggfunc='size', fill_value=0)
    return matrix

#main_df = pd.read_csv('your_main_dataframe.csv')  # Replace with your actual main dataframe

matrix1 = process_aitool_column(main_df, 'AIToolInterested in Using', 'Interested In Using')
matrix2 = process_aitool_column(main_df, 'AIToolCurrently Using', 'Currently Using')
matrix3 = process_aitool_column(main_df, 'AIToolNot interested in Using', 'Not Interested Using')

concatenated_df = pd.concat([matrix1, matrix2, matrix3], axis=0, ignore_index=False)
#concatenated_df.to_csv('merged_aitools_cnct1.csv')

## stacked bar

In [ ]:
'AINextVery different',
       'AINextNeither different nor similar', 'AINextSomewhat similar',
       'AINextVery similar', 'AINextSomewhat different'

In [118]:
#main_df = pd.read_csv('your_main_dataframe.csv')  # Replace with your actual main dataframe
def process_aitool_column(df, column_name, tool_category):
    df_temp = df[[column_name]].copy()
    df_temp[tool_category] = df_temp[column_name].str.split(';')
    df_temp = df_temp.explode(tool_category).dropna()
    df_temp['Ai Next'] = tool_category
    df_temp = df_temp.drop(column_name, axis=1)
    matrix = df_temp.pivot_table(index='Ai Next', columns=tool_category, aggfunc='size', fill_value=0)
    return matrix




In [130]:
matrix1 = process_aitool_column(main_df, 'AINextVery different', 'Very different')
matrix2 = process_aitool_column(main_df, 'AINextSomewhat similar', 'Somewhat similar')
matrix3 = process_aitool_column(main_df, 'AINextVery similar', 'Very similar')
matrix4 = process_aitool_column(main_df, 'AINextSomewhat different', 'Somewhat different')
matrix5 = process_aitool_column(main_df, 'AINextNeither different nor similar', 'Neither different nor similar')


In [131]:
ainext_concatenated_df = pd.concat([matrix1, matrix2, matrix3,matrix4,matrix5], axis=0, ignore_index=False)




In [133]:
ainext_concatenated_df

ainext_concatenated_df.to_csv('ainext_concatenated.csv')


## Bubble Chart ZOOM d3 

In [6]:
c_df = main_df

In [7]:
from collections import Counter 
def parse(col):
    bigtxt = ";".join(c_df[col].dropna())
    wrds = bigtxt.split(";")
    wrds = Counter(wrds).most_common()
    resp = {'name' : col, "children" : [], "size":""}
    for wrd in wrds:
        doc = {'name' : wrd[0], "size": wrd[1]}
        resp['children'].append(doc)
    return resp
results = {'name' : 'flare', "children" : [], "size":""}
AIDevHaveWorkedWith = parse('LanguageHaveWorkedWith')
results['children'].append(AIDevHaveWorkedWith)

AIDevWantToWorkWith = parse('DatabaseHaveWorkedWith')
results['children'].append(AIDevWantToWorkWith)

AISearchWantToWorkWith = parse('PlatformHaveWorkedWith')
results['children'].append(AISearchWantToWorkWith)

AIToolInterested_Using = parse('WebframeHaveWorkedWith')
results['children'].append(AIToolInterested_Using)

AISearchHaveWorkedWith = parse('OfficeStackAsyncHaveWorkedWith')
results['children'].append(AISearchHaveWorkedWith)

AIToolCurrently_Using = parse('NEWCollabToolsHaveWorkedWith')
results['children'].append(AIToolCurrently_Using)



In [8]:
with open('ai_tool_Bubble.json', 'w') as outfile:  
    json.dump(results, outfile)